# Detección de movimientos fraudulentos
##  Análisis de  284.807 Tárjetas de Crédito

El siguiente caso de estudio es un conjunto de 284.807 entradas de movimientos en las tarjetas de crédito. En estos datos existe un pequeño porcentaje que son movimiento fraudulentos de modo que me dispongo a analizarlo utilizando algoritmos como **Xgboost, Red Neuronal Artificial, Teorema de Bayes, Regresión Logística, Soporte Vectorial, etc.**

link data: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud

**Quiero destacar que todos estos datos están pseudonimizados, menos 3 columnas necesarias (de las 31) que seria el tiempo, el importe del movimiento y el tipo de transacción que se realiza (1 si es fraude y 0 si no lo es)**

### Librerías

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

### Clase Algoritmos
Esta clase se encargará de calcular la precisión de los algoritmos siempre y cuando inicialice correctamente el constructor con sus parametros correctos. Después solo hay que llamar a la funcion **precision()**. También almacenará los datos de precisión en un dataframe **df** para una consulta posterior en el apartado de la conclusión.

In [2]:
#-------------------------------------------------------------------------------------
datos = {'Nombre_Algoritmo':[], 'Precision':[]}  
df = pd.DataFrame(data=datos) 
# -------------------------------------------------------------------------------------
class Algoritmos:
    
    def __init__(self,nombre,cm, y_test):
        self.nombre = nombre
        self.cm = cm
        self.y_test = y_test
        
    def __str__(self):
        return (self.nombre)
        
    def precision(self):
        global df
        assert(self.cm.shape==(2,2))
        assert(type(self.y_test) == pd.DataFrame)
        assert(type(self.cm[0][0]) == np.int64)
        precision_02 = ((self.cm[0][0] + self.cm[1][1])/len(self.y_test)) * 100
        assert(type(precision_02) == np.float64)
        df = df.append({'Nombre_Algoritmo':self.nombre, 'Precision':precision_02}, ignore_index=True)
        return("%.2f" %precision_02)
        
#-----------------------------------------------------------------------------------        

###  Carga y Visualización de datos

In [3]:
# PATH
path = "C:/Users/USUARIO/Desktop/DataTarjetaCredit/"
data = pd.read_csv(path + "creditcard.csv")

In [4]:
data # Visualización de datos

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [5]:
# Visualizo cuantos movimimientos fraudulentos hay. 
p = data['Class'] == 1
filtro = data[p]
len(filtro.index) # 492 fraudulentos de 284.807 movimientos

492

In [6]:
data.describe() 
# Si compruebo la media y las desviaciones se observa a simple vista que hay 
# una alta desviación, por ejemplo en la columna "Amount (importe) con un 
# max de '25691.16000' cuando su media es de 88 y la desviacion del 250.
# También existe una gran desviación en los valores V1-V28 pero como son
# valores pseudonimizados hasta ahí puedo observar"

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


### Preprocesado de datos

Primero compruebo si hay algun valor nulo en mi conjunto de datos. Luego observo el tipo de datos por si existiera alguna columna tipo object para transformarla. En este caso no es necesario ya que todas son entradas de datos tipo float64 y una int64. 

In [7]:
# Preprocesado
data.isnull().sum() # ningun dato nulo

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [8]:
data.dtypes # Tipo de datos

Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       int64
dtype: object

### Transformacíon de datos

Almaceno las columnas que considero variables independientes y dependiente, por ejemplo en este supuesto todas las columnas son importantes en el problema de modo que no me deshago de ninguna. El siguiente paso es considerar quien es la variable dependiente y las variables independientes:

- El objetivo de este supesto es predecir la columna **'Class'** para pronosticar si un movimiento es fraudulento  o no . De modo que la columna **'Class' será la variable dependiente**.

- Como he dicho antes, todas las columnas tienen relevancia en este problema entonces **las variables independientes seran todas las columnas menos 'Class'**

In [9]:
X = data.iloc[:, 0:-1]
Y = data.iloc[:, -1:]

In [10]:
X

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00


In [11]:
Y

,Class
0,0
1,0
2,0
3,0
4,0
...,...
284802,0
284803,0
284804,0
284805,0


### TRAIN Y TEST
Después de visualizar los datos, preprocesarlos, considerar que columnas son relevantes para el caso de estudio y dividirlos en dos matrices ahora ya se pasaría a la fase de división de datos, es decir, considerar cuanto porcentaje de datos del dataset sería para entrenar el modelo, otro porcentaje para testearlo y verificar su precisión. Normalmente:

- **TRAIN**, tiene el **80% de datos del dataset original** ya que cuanto más datos más comprobaciones realizará y más fiable será.
- **TEST**, tiene el **20% de datos del dataset original** y con él se mide la precisión.

Como sería un proceso muy laborioso hacerlo manualmente con está gran cantidad de datos hay una función en Python del modulo **sklearn** llamada **train_test_split** que nos facilita esta división de datos.

In [12]:
# Divido los datos usando la funcion train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=(0))

# Escalado de variables
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Train tiene 227.845 datos para entrenar
# Test tiene 56.926 datos para testear nuestros algoritmos

In [13]:
p = y_test['Class'] == 1
filtro = y_test[p]
len(filtro.index) # Hay un total de 101 movimientos fraudulentos en los datos para testear el modelo

101

### Algoritmo Regresión Logística
La regresión logística es un tipo de análisis de regresión utilizado para predecir el resultado de una variable categórica (una variable que puede adoptar un número limitado de categorías) en función de las variables independientes o predictoras.

In [14]:
LogisticR = LogisticRegression(random_state=0) # random state = 0
LogisticR.fit(X_train, y_train) #Ajustamos el modelo

# Realizamos la predicción de los resultados con el conjunto de testing
y_pred = LogisticR.predict(X_test)

# Matriz de confusion
# De los 56.926 datos  entradas de clientes
cm = confusion_matrix(y_test, y_pred) 
cm

array([[56852,     9],
       [   37,    64]], dtype=int64)

In [15]:
cm[1][1] # Tiene una precisión del 99,92%, ha acertado 64 de 101 en transacciones fraudulentas del testeo de 56.926 datos

64

In [16]:
Reg = Algoritmos("Regresión Logística", cm, y_test)
Reg.precision() # Precisión del 99.92%

'99.92'

### Algoritmo KNN (K Vecinos más cercanos)

Este algoritmo es un clasificador de aprendizaje supervisado no paramétrico, que utiliza la proximidad para hacer clasificaciones o predicciones sobre la agrupación de un punto de datos individual. Si bien se puede usar para problemas de regresión o clasificación, generalmente se usa como un algoritmo de clasificación, partiendo de la suposición de que se pueden encontrar puntos similares cerca uno del otro.

In [17]:
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2) # Objeto clasifier
KNN.fit(X_train, y_train)

y_pred = KNN.predict(X_test)
cm = confusion_matrix(y_test, y_pred) #  99.95% de precision
cm

array([[56852,     9],
       [   20,    81]], dtype=int64)

In [18]:
cm[1][1] # Tiene una precisión del 99.95%, ha acertado 81 de 101 en transacciones fraudulentas del testeo de 56.926 datos

81

In [19]:
KNN = Algoritmos("KNV", cm, y_test) 
KNN.precision()

'99.95'

### Algoritmo Teorema de Bayes
El teorema de Bayes es utilizado para calcular la probabilidad de un suceso, teniendo información de antemano sobre ese suceso. Podemos calcular la probabilidad de un suceso A, sabiendo además que ese A cumple cierta característica que condiciona su probabilidad. El teorema de Bayes entiende la probabilidad de forma inversa al teorema de la probabilidad total. El teorema de la probabilidad total hace inferencia sobre un suceso B, a partir de los resultados de los sucesos A. Por su parte, Bayes calcula la probabilidad de A condicionado a B.

In [20]:
BAYES = GaussianNB()
BAYES.fit(X_train, y_train)

y_pred = BAYES.predict(X_test)
cm = confusion_matrix(y_test, y_pred) #97.82 % de precision
cm

array([[55636,  1225],
       [   15,    86]], dtype=int64)

In [21]:
cm[1][1]  # Tiene una precisión del 97.82%, ha acertado 86 de 101 en transacciones fraudulentas del testeo de 56.926 datos
# pero tiene demasiados falsos positivos 1225

86

In [22]:
TeorBayes = Algoritmos("Teorema de Bayes", cm, y_test)
TeorBayes.precision()

'97.82'

### Árbol de Clasificación
Un árbol de clasificación es un tipo de árbol de decisiones. Utiliza la medida de impurezas de Gini para clasificar los registros en las categorías del campo objetivo. Las predicciones se basan en combinaciones de valores en los campos de entrada.
Un árbol de clasificación calcula la categoría de destino pronosticada para cada nodo en el árbol.

In [23]:
DecisionTree = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
DecisionTree.fit(X_train, y_train)

y_pred = DecisionTree.predict(X_test)
cm = confusion_matrix(y_test, y_pred)  # 99,92% de precision
cm

array([[56836,    25],
       [   23,    78]], dtype=int64)

In [24]:
cm[1][1] # Tiene una precisión del 99,92%, ha acertado 78 de 101 en transacciones fraudulentas del testeo de 56.926 datos

78

In [25]:
Arbol = Algoritmos("Árboles de Clasificación", cm, y_test)
Arbol.precision()

'99.92'

In [26]:
# Aleatorio Entropia
rand = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
rand.fit(X_train, y_train)

y_pred = rand.predict(X_test)
cm = confusion_matrix(y_test, y_pred) # 99.95% precision
cm

array([[56854,     7],
       [   24,    77]], dtype=int64)

In [27]:
cm[1][1]  # Tiene una precisión del 99.95%, ha acertado 77 de 101 en transacciones fraudulentas del testeo de 56.926 datos

77

In [28]:
ArbolRandomEntropia = Algoritmos("Árboles Aleatorios Entropia", cm, y_test)
ArbolRandomEntropia.precision()

'99.95'

In [29]:
# Aleatorio Gini
rand02 = RandomForestClassifier(n_estimators = 10, criterion = 'gini', random_state = 0)
rand02.fit(X_train, y_train)

y_pred = rand02.predict(X_test)
cm = confusion_matrix(y_test, y_pred) # 99.94%% precision
cm

array([[56855,     6],
       [   26,    75]], dtype=int64)

In [30]:
cm[1][1]  # Tiene una precisión del 99.94%, ha acertado 75 de 101 en transacciones fraudulentas del testeo de 56.926 datos

75

In [31]:
ArbolRandomGini = Algoritmos("Árboles Aleatorios Gini", cm, y_test)
ArbolRandomGini.precision()

'99.94'

### Algoritmo de XGBOOST
XGBoost es una biblioteca de aumento de gradiente distribuida optimizada diseñada para ser altamente eficiente , flexible y portátil . Implementa algoritmos de aprendizaje automático bajo el marco Gradient Boosting . XGBoost proporciona un impulso de árbol paralelo (también conocido como GBDT, GBM) que resuelve muchos problemas de ciencia de datos de una manera rápida y precisa

In [32]:
XGBOOST = XGBClassifier()
XGBOOST.fit(X_train, y_train)

y_pred = XGBOOST.predict(X_test)
cm = confusion_matrix(y_test, y_pred) # % precision
cm

array([[56855,     6],
       [   17,    84]], dtype=int64)

In [33]:
cm[1][1] # Tiene una precisión del 99.96%, ha acertado 84 de 101 en transacciones fraudulentas del testeo de 56.926 datos

84

In [34]:
xgb = Algoritmos("XGBOOST", cm, y_test)
xgb.precision()

'99.96'

### Algoritmo Red Neuronal Artificial
Una red neuronal es un modelo simplificado que emula el modo en que el cerebro humano procesa la información: Funciona simultaneando un número elevado de unidades de procesamiento interconectadas que parecen versiones abstractas de neuronas.
La red aprende examinando los registros individuales, generando una predicción para cada registro y realizando ajustes a las ponderaciones cuando realiza una predicción incorrecta. Este proceso se repite muchas veces y la red sigue mejorando sus predicciones hasta haber alcanzado uno o varios criterios de parada.

In [35]:
# Construir la red neuronal artificial con keras del modelo secuencial
RNA = tf.keras.models.Sequential()

# 1º Capa de inputs
RNA.add(tf.keras.layers.Dense(units=10, activation='relu'))
# 2º Capa oculta de 10 nodos
RNA.add(tf.keras.layers.Dense(units=10, activation='relu'))
# Ultima capa de salidad, con funcion sigmoide
RNA.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

RNA.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
RNA.fit(X_train, y_train, batch_size = 100, epochs = 200)

Epoch 1/200
2279/2279 [==============================] - 6s 2ms/step - loss: 0.0219 - accuracy: 0.9984
Epoch 2/200
2279/2279 [==============================] - 6s 2ms/step - loss: 0.0040 - accuracy: 0.9991
Epoch 3/200
2279/2279 [==============================] - 5s 2ms/step - loss: 0.0034 - accuracy: 0.9993
Epoch 4/200
2279/2279 [==============================] - 5s 2ms/step - loss: 0.0032 - accuracy: 0.9994
Epoch 5/200
2279/2279 [==============================] - 6s 2ms/step - loss: 0.0030 - accuracy: 0.9994
Epoch 6/200
2279/2279 [==============================] - 5s 2ms/step - loss: 0.0028 - accuracy: 0.9994
Epoch 7/200
2279/2279 [==============================] - 5s 2ms/step - loss: 0.0027 - accuracy: 0.9994
Epoch 8/200
2279/2279 [==============================] - 6s 2ms/step - loss: 0.0025 - accuracy: 0.9994
Epoch 9/200
2279/2279 [==============================] - 5s 2ms/step - loss: 0.0025 - accuracy: 0.9994
Epoch 10/200
2279/2279 [==============================] - 5s 2ms/step - l

2279/2279 [==============================] - 5s 2ms/step - loss: 0.0011 - accuracy: 0.9997
Epoch 158/200
2279/2279 [==============================] - 6s 2ms/step - loss: 0.0011 - accuracy: 0.9997
Epoch 159/200
2279/2279 [==============================] - 5s 2ms/step - loss: 0.0012 - accuracy: 0.9997
Epoch 160/200
2279/2279 [==============================] - 5s 2ms/step - loss: 0.0011 - accuracy: 0.9997
Epoch 161/200
2279/2279 [==============================] - 6s 2ms/step - loss: 0.0011 - accuracy: 0.9997
Epoch 162/200
2279/2279 [==============================] - 5s 2ms/step - loss: 0.0011 - accuracy: 0.9997
Epoch 163/200
2279/2279 [==============================] - 5s 2ms/step - loss: 0.0011 - accuracy: 0.9997
Epoch 164/200
2279/2279 [==============================] - 6s 2ms/step - loss: 0.0011 - accuracy: 0.9997
Epoch 165/200
2279/2279 [==============================] - 5s 2ms/step - loss: 0.0011 - accuracy: 0.9997
Epoch 166/200
2279/2279 [==============================] - 5s 2ms/ste

In [36]:
y_pred = RNA.predict(X_test)
y_pred_02 = (y_pred > 0.5)
y_pred_02 = np.where(y_pred_02=='True',1,y_pred_02)
cm = confusion_matrix(y_test, y_pred_02)    # 
cm

array([[56852,     9],
       [   24,    77]], dtype=int64)

In [37]:
cm[1][1]

77

In [38]:
redneuro = Algoritmos("Red Neuronal Artificial", cm, y_test)
redneuro.precision()

'99.94'

### Conclusión

Estas son las precisiones que he sacado de mis algoritmo después de testearlos con datos que no conocían. Aunque son precisiones muy parecidas, **la mejor es el algorimto XGBOOST con 99.96% de precisión.**

In [40]:
 df.sort_values(by='Precision',ascending=False)

,Nombre_Algoritmo,Precision
6,XGBOOST,99.959622
1,KNV,99.949089
4,Árboles Aleatorios Entropia,99.945578
5,Árboles Aleatorios Gini,99.943822
7,Red Neuronal Artificial,99.942067
0,Regresión Logística,99.919244
3,Árboles de Clasificación,99.915733
2,Teorema de Bayes,97.823110


El algoritmo de **Xgboost** tiene la siguiente matriz de confusión:
- **array([[56855,     6],[   17,    84]],)**, que se traduciría a que ha detectado **84 movimientos fraudulentos de 101**, con **precisión 99.959622 %**.

Por otro lado, el **teorema de Bayes** ha sacado más movimientos fraudulentos que este algoritmo anterior, a continuación su matriz de confusión:

- **array([[55636,  1225],[   15,    86]],)** que se traduciría a que ha detectado **86 movimientos fraudulentos de 101**, con **precisión 97.823110 %**.

Entonces, **¿cúal es mejor?**. Obviamente uno pensaría que el Teorema de Bayes es mejor que Xgboost porque "acierta" más transacciones fraudulentas que Xgboost pero también tiene muchos más falsos positivos-negativos que Xgboost.

**A nivel de eficacia Xgboost solo se equivoca en 23 datos de 56.926 del total de transacciones mientras que Bayes se equivoca en 1240 datos, por tanto el mejor algoritmo para pronosticar/analizar nuevos datos de transacciones fraudulentas con estás características sería Xgboost con 99.96% de precisión.**


**Autor: Carlos Mir Martínez**

**Fecha: 15/06/2022**